In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

sys.path.insert(0, "../")

import warnings
warnings.filterwarnings('ignore')

from michael import janet

2022-10-10 12:50:06.581479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Hi there. Welcome to the good place.


In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

In [3]:
index = 1

In [4]:
j = janet.boot(uni, index=index, use_prior=False)
j.prepare_data()

Already have data downloaded for Gaia ID 2174289161681454720.
If you want to check for new data, run `janet.update()`.
Removing 186 bad data points (out of 1121) using the TESS provided QUALITY array
Summing over 2 x 2 pixel lightcurves. Weighting=median
Removing 4 bad data points (out of 1130) using the TESS provided QUALITY array
Summing over 2 x 2 pixel lightcurves. Weighting=median


Running pixels in aperture: 100%|████████████| 300/300 [00:01<00:00, 288.82it/s]


In [5]:
j.get_rotation(period_range=(1, 10))

### Running Simple Astropy Lomb-Scargle on Sector 16-17 on star 2174289161681454720 ###
### Completed Simple Astropy Lomb-Scargle for Sector 16-17 on star 2174289161681454720 ###
### Running Wavelet Estimation for Sector 16-17 on star 2174289161681454720 ###


100%|█████████████████████████████████████████| 152/152 [00:02<00:00, 73.18it/s]

### Completed Wavelet Estimation for Sector 16-17 on star 2174289161681454720 ###
### Running Composite ACF estimation for Sector 16-17 on star 2174289161681454720 ###
### Completed Composite ACF estimation for Sector 16-17 on star 2174289161681454720 ###
### Running ACF Estimation on star 2174289161681454720 ###
### Completed ACF Estimation for Sector 16-17 on star 2174289161681454720 ###


In [6]:
j.validate_rotation()
j.view()

AttributeError: 'Series' object has no attribute 'drpona'

In [14]:
ebest[['e_CACF']].values

array([0.3296763681717397], dtype=object)

In [22]:
j.results.loc['best']

SLS               3.535797
e_SLS             0.325392
h_SLS             0.000018
f_SLS                  0.0
SW                3.480956
e_SW              0.877798
h_SW              1.057789
CACF              8.707782
e_CACF            0.329676
h_CACF            0.061869
ACF               1.854169
e_ACF                  NaN
p2p_SLS           0.002119
f_p2p_SLS              NaN
p2p_SW            0.002055
f_p2p_SW               NaN
p2p_CACF          0.002724
f_p2p_CACF             NaN
p2p_ACF           0.001345
f_p2p_ACF              NaN
s_SLS                16-17
s_SW                 16-17
s_CACF               16-17
s_ACF                16-17
overall           3.535797
e_overall         0.325392
method_overall         SLS
Name: best, dtype: object

In [24]:
j.results

,SLS,e_SLS,h_SLS,f_SLS,SW,e_SW,h_SW,CACF,e_CACF,h_CACF,...,f_p2p_CACF,p2p_ACF,f_p2p_ACF,s_SLS,s_SW,s_CACF,s_ACF,overall,e_overall,method_overall
16-17,3.535797,0.325392,0.000018,0.0,3.480956,0.877798,1.057789,8.707782,0.329676,0.061869,...,0.0,0.001345,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
best,3.535797,0.325392,0.000018,0.0,3.480956,0.877798,1.057789,8.707782,0.329676,0.061869,...,NaN,0.001345,NaN,16-17,16-17,16-17,16-17,3.535797,0.325392,SLS


In [25]:
    # Validate the best estimates against one another
    # Check to see if they agree closely with one another
    methods = [['SLS', 'SW', 'CACF', 'ACF']]
    best = j.results.loc['best', ['SLS','SW','CACF', 'ACF']]#.dropna()
    ebest = j.results.loc['best', ['e_SLS','e_SW','e_CACF', 'e_ACF']]#.dropna()
    p2ps = j.results.loc['best', ['p2p_SLS','p2p_SW','p2p_CACF', 'p2p_ACF']]#.dropna()']
    j.results['f_overall'] = np.zeros(len(j.results)).astype(int)

    # If they agree, then pick the one with the highest p2p value
    if np.abs(np.diff(best.dropna(), len(best.dropna())-1)) < np.sqrt(np.nansum(ebest**2)):
        s = np.argmax(p2ps)
        j.results.loc['best', 'overall'] = best[s]
        j.results.loc['best', 'e_overall'] = ebest[s]
        j.results.loc['best', 'method_overall'] = methods[s]

    # If they disagree, see if two of them are in agreement
    else:
        # We check in this order, as the priority is CACF -> SW -> SLS -> ACF
        d = np.abs(np.diff(best[['SLS','SW']])) < np.sqrt(np.sum(ebest[['e_SLS', 'e_SW']]**2))
        c = np.abs(np.diff(best[['ACF', 'CACF']])) < ebest[['e_CACF']].values
        b = np.abs(np.diff(best[['SLS','CACF']])) < np.sqrt(np.sum(ebest[['e_SLS', 'e_CACF']]**2))
        a = np.abs(np.diff(best[['SW','CACF']])) < np.sqrt(np.sum(ebest[['e_SW', 'e_CACF']]**2))

        # SW and CACF are in agreement
        if a:
            s = np.argmax(p2ps[['p2p_SW','p2p_CACF']])
            j.results.loc['best', 'overall'] = best[['SW', 'CACF']][s]
            j.results.loc['best', 'e_overall'] = ebest[['e_SW', 'e_CACF']][s]
            j.results.loc['best', 'method_overall'] = ['SW','CACF'][s]
            j.results.loc['best', 'f_overall'] += int(4)

        # SLS and CACF are in agreement
        elif b:
            s = np.argmax(p2ps[['p2p_SLS','p2p_CACF']])
            j.results.loc['best', 'overall'] = best[['SLS', 'CACF']][s]
            j.results.loc['best', 'e_overall'] = ebest[['e_SLS', 'e_CACF']][s]
            j.results.loc['best', 'method_overall'] = ['SLS','CACF'][s]
            j.results.loc['best', 'f_overall'] += int(4)

        # CACF and ACF are in agreement
        elif c:
            s = np.argmax(p2ps[['p2p_ACF','p2p_CACF']])
            j.results.loc['best', 'overall'] = best[['ACF', 'CACF']][s]
            j.results.loc['best', 'e_overall'] = ebest[['e_ACF', 'e_CACF']][s]
            j.results.loc['best', 'method_overall'] = ['ACF','CACF'][s]
            j.results.loc['best', 'f_overall'] += int(4)

        # SLS and SW are in agreement
        elif d:
            s = np.argmax(p2ps[['p2p_SLS','p2p_SW']])
            j.results.loc['best', 'overall'] = best[['SLS', 'SW']][s]
            j.results.loc['best', 'e_overall'] = ebest[['e_SLS', 'e_SW']][s]
            j.results.loc['best', 'method_overall'] = ['SLS','SW'][s]
            j.results.loc['best', 'f_overall'] += 4

        # There is no agreement whatsover
        else:
            if np.isfinite(best['CACF']):
                j.results.loc['best', 'overall'] = best['CACF']
                j.results.loc['best', 'e_overall'] = ebest['e_CACF']
                j.results.loc['best', 'f_overall'] += 1
                warnings.warn("No estimates could agree. Please inspect the results carefully yourself.")

            elif np.isfinite(best['SW']):
                frac = ebest[['e_SLS','e_SW']].values /  best[['SLS','SW']].values
                s = np.argmin(frac)
                j.results.loc['best', 'overall'] = best[['SLS','SW']][s]
                j.results.loc['best', 'e_overall'] = ebest[['e_SLS','e_SW']][s]
                j.results.loc['best', 'f_overall'] += 2
                warnings.warn("No estimates could agree. Please inspect the results carefully yourself.")

In [47]:
    # Check if any individual sectors are wildly out of line, possibly due to binaries
    res = j.results.loc[(j.results.index != 'best') & (j.results.index != 'all'), ['SLS','SW','CACF', 'ACF']]
    err = j.results.loc[(j.results.index != 'best') & (j.results.index != 'all'), ['e_SLS', 'e_SW', 'e_CACF', 'e_ACF']]

    a = np.abs(np.diff(res,axis=0, n = len(res)-1))
    b = np.sqrt(np.nansum(err**2, axis=0))

    # Do any sectors disagree repeatedly over 1 sigma across all sectors?
    if all(list((a-b > 0)[0])):
        j.results.loc['best', 'f_overall'] += 8

KeyError: 0

In [46]:
np.sum(err**2, axis=0)

e_SLS     0.105880
e_SW      0.770529
e_CACF    0.108687
e_ACF     0.000000
dtype: float64

In [36]:
a = np.abs(np.diff(res,axis=0, n = len(res)-1))
a

,SLS,SW,CACF,ACF
16-17,3.535797,3.480956,8.707782,1.854169


In [ ]:
    # Validate the best estimates against one another
    # Check to see if they agree closely with one another
    best = j.results.loc['best', ['SLS','SW','CACF', 'ACF']]#.dropna()
    ebest = j.results.loc['best', ['e_SLS','e_SW','e_CACF', 'e_ACF']]#.dropna()
    p2ps = j.results.loc['best', ['p2p_SLS','p2p_SW','p2p_CACF', 'p2p_ACF']]#.dropna()']

In [ ]:
p2ps

In [ ]:
np.abs(np.diff(best.dropna(), len(best.dropna())-1)) < np.sqrt(np.nansum(ebest**2))



In [ ]:
np.diff(best.dropna(), len(best.dropna())-1)

In [ ]:
np.diff(best, n=3)

In [ ]:
j.results

Plot lightcurves

In [ ]:
ax = j.void['clc_16-17'].plot(label='eleanor')

pls = ['c','raw','pca','corn','r', 'cpm']
for pl in pls:
    j.void[f'{pl}lc_16-17'].plot(ax=ax, label=pl)
ax.legend(fontsize=10)

In [ ]:
ax = j.void['clc_16-17'].to_periodogram(maximum_period=27).plot(label='eleanor', view='period')

pls = ['c','raw','pca','corn','r', 'cpm']
for pl in pls:
    j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).plot(ax=ax, label=pl,view='period', scale='log')
    ax.axvline(j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).period_at_max_power.value)
    print(j.void[f'{pl}lc_16-17'].to_periodogram(maximum_period=27).period_at_max_power)
ax.legend(fontsize=10)